In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from tqdm._tqdm_notebook import tqdm, tqdm_notebook, tnrange
from S3_read_write import load_df_s3, save_df_s3

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
tqdm_notebook.pandas('Progress')

# Load Amazon Reviews Data

In [3]:
bucket_name = 'amazon-reviews-project'

In [30]:
reviews = load_df_s3(bucket_name, 'amazon_reviews/reviews_data_clean', filetype='text', sep='|')

In [31]:
reviews.shape    # 585,444 records

(585444, 8)

In [32]:
reviews.head()

,asin,helpful,reviewText,overall,summary,description,title,categories_clean
0,0929619730,"[0, 0]",B-flax-D is a re...,5.0,Dpes the job well,Contains Organic...,New Generation B...,Health & Persona...
1,0978559088,"[1, 1]",Studies show tha...,4.0,"Fast shipping, g...",Everyone knows t...,Nutrihill Resver...,Health & Persona...
2,0978559088,"[1, 1]",I started taking...,5.0,Bioavailability ...,Everyone knows t...,Nutrihill Resver...,Health & Persona...
3,0978559088,"[0, 1]",I tried Nutrihil...,1.0,Other Resveratro...,Everyone knows t...,Nutrihill Resver...,Health & Persona...
4,0978559088,"[0, 0]",I really liked t...,5.0,I can't find thi...,Everyone knows t...,Nutrihill Resver...,Health & Persona...


In [33]:
reviews.dtypes

asin                 object
helpful              object
reviewText           object
overall             float64
summary              object
description          object
title                object
categories_clean     object
dtype: object

## Data Cleaning

In [52]:
reviews.categories_clean.unique()[:10]

array(['Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, Resveratrol',
       'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multivitamins',
       'Health & Personal Care, Vitamins & Dietary Supplements, Vitamins, Vitamin B, B3 (Niacin)',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements, Green Tea',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements, Green Coffee Bean Extract',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, CoQ10',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal

The catergories' list indicates that there may be some reviews in the dataset unrelated to health supplements.  Let's get rid of these.

In [35]:
reviews[reviews.categories_clean.str.contains('CDs & Vinyl')].title.unique()

array(['Liturgy of St. John Chrysostom', 'Origins',
       'Sounds of the Earth: Soft Ocean Sounds', 'Bali',
       'Tranquil Waters', 'Bach: St. John Passion, BWV 245',
       '21st Century Soul', 'Bodies for Strontium', "John's Bunch",
       'An Evening of Paganini', "John's Other Bunch",
       'Sus Mas Grandes Exitos', 'Complex Simplicity',
       'Kidnapped By Neptune', 'Roman Chant / Easter Vespers', 'Dead 60s',
       "Cilla in the 60's", 'Chromium', 'Letters From the Vitamin Sea',
       'The Stinging Nettles', 'Tendres Annees 60', 'Wehiwehi Hawaii',
       'none'], dtype=object)

In [36]:
len(reviews[reviews.categories_clean.str.contains('CDs & Vinyl')])

263

The product titles shown above are all music albums/songs.

In [37]:
reviews_filt = reviews[~(reviews.categories_clean.str.contains('CDs & Vinyl'))]   # remove rows with category including 'CDs & Vinyl'

In [51]:
reviews_filt.categories_clean.unique()[:10]

array(['Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, Resveratrol',
       'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multivitamins',
       'Health & Personal Care, Vitamins & Dietary Supplements, Vitamins, Vitamin B, B3 (Niacin)',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements, Green Tea',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements, Green Coffee Bean Extract',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, CoQ10',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal

In [39]:
reviews_filt[reviews_filt.categories_clean.str.contains('Software')]

,asin,helpful,reviewText,overall,summary,description,title,categories_clean
3639,B00009QP4Q,"[2, 2]",The company has ...,5.0,lives up to its ...,Alpha Five's QLi...,none,Health & Persona...
50015,B0002TIEQQ,"[0, 0]",I ordered this f...,1.0,waste of money,Self help tutori...,none,Health & Persona...


In [40]:
reviews_filt = reviews_filt[~(reviews_filt.categories_clean.str.contains('Software'))]

In [41]:
len(reviews_filt)

585179

In [53]:
# Get rid of reviews of pet-related products
search_for = [' pet ', ' cat ', ' dog ']
pattern = '|'.join(search_for)
reviews_filt.title.str.contains(pattern, case=False).sum()

277

In [50]:
reviews_filt[reviews_filt.title.str.contains(pattern, case=False)]['title'].values[:10]

array(['Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'Composure Liquid for Dogs and Cat (188 SERVINGS)'], dtype=object)

In [54]:
# Get rid of all pet products
reviews_filt = reviews_filt[~(reviews_filt.title.str.contains(pattern, case=False))]

In [55]:
# saving the cleaned dataframe
save_df_s3(df=reviews_filt, bucket_name=bucket_name, filepath='amazon_reviews/reviews_data_clean_v2.feather')

In [56]:
reviews_filt.asin.nunique()     # 48,535 unique products and 585,179 reviews

48501

## Examine One Observation

In [57]:
example = reviews_filt.iloc[0]

In [58]:
example.asin     # Amazon Standard Identification Number

'0929619730'

In [59]:
example.title     # this is the product's name

'New Generation B-Flax-D'

In [60]:
example.categories_clean   # previously filtered/curated categories of interest

'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements'

In [61]:
example.description       # product description provided by the seller

'Contains Organic Cold-Milled Flaxseed\nValuable source of soluble and insoluble fiber\nProvides Omega-3 essential fats, and many other nutrients to help achieve and maintain optimal bowel function.\n\nContains Vitamin B12\nB12 helps prevent nerve damage\nB12 aids in healthy cell formation.\nB12 helps prevent anemia\n\nContains Vitamin D\nVitamin D assists the body in the absorption of important minerals like calcium.\n\nContains Seleno-yeast\nA source of selenium, a mineral with powerful anti-viral and disease-fighting properties.\n\nContains Vitamin K2\nMenaQ7TM provides vitamin K2 (menaquinone), extracted and concentrated from natto without solvents. Vitamin K2 prevents arterial calcification and promotes strong bones by improving cross-linking of osteocalcin, a protein found in bones. The amount here has been clinically shown not to interfere with blood anti-coagulant medication. \n\nServing Size:\n1/4 Cup (30 Grams)\n\nServings Per Container:\n30 Servings per container\n\nNet Wt. 

In [62]:
example.summary      # review title

'Dpes the job well'

In [63]:
example.reviewText   # review content

'B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt around. Good product, good price, good results.'

Here's what the actual review looks like:

In [64]:
example.overall     # the rating provided by the reviewer

5.0

In [29]:
example.helpful

'[0, 0]'

In [27]:
Image(url= "images/amazon_review_screenshot.png")

# Data Pre-processing

Let us start off using only the title (`summary`) and body (`reviewText`) of each review.

In [4]:
%%time
df = load_df_s3(bucket_name, filepath='amazon_reviews/reviews_data_clean_v2.feather', filetype='feather')

CPU times: user 3.84 s, sys: 3 s, total: 6.85 s
Wall time: 20.1 s


In [5]:
df.dtypes

asin                 object
helpful              object
reviewText           object
overall             float64
summary              object
description          object
title                object
categories_clean     object
dtype: object

In [6]:
df.drop(['helpful', 'overall', 'title', 'categories_clean', 'description'], axis=1, inplace=True)

In [7]:
df.head()

,asin,reviewText,summary
0,0929619730,B-flax-D is a regular at our house. It does it...,Dpes the job well
1,0978559088,Studies show that Resveratrol is poorly absorb...,"Fast shipping, good communication"
2,0978559088,I started taking this after both my parents di...,Bioavailability is the key
3,0978559088,"I tried Nutrihill, but did not feel any of the...",Other Resveratrol Supplements are Better
4,0978559088,I really liked this product because it stayed ...,"I can't find this product any longer, and I wi..."


In [8]:
# for each review, concatenate the review title and body
df.reviewText = df.summary + '. ' + df.reviewText

In [9]:
pd.set_option('max_colwidth', 200)
df.head()

,asin,reviewText,summary
0,0929619730,"Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt ...",Dpes the job well
1,0978559088,"Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This ...","Fast shipping, good communication"
2,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...,Bioavailability is the key
3,0978559088,"Other Resveratrol Supplements are Better. I tried Nutrihill, but did not feel any of the supposed health benefits. I started reading and realized that even though buccal delivery is the best, the ...",Other Resveratrol Supplements are Better
4,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of...","I can't find this product any longer, and I wish I could."


Let's drop the `summary` column now:

In [10]:
df.drop(['summary'], axis=1, inplace=True)

In [11]:
df.head()

,asin,reviewText
0,0929619730,"Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt ..."
1,0978559088,"Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This ..."
2,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...
3,0978559088,"Other Resveratrol Supplements are Better. I tried Nutrihill, but did not feel any of the supposed health benefits. I started reading and realized that even though buccal delivery is the best, the ..."
4,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of..."


In [12]:
pd.set_option('max_colwidth', 20)

## Remove Missing Reviews

In [13]:
df.reviewText.isnull().sum()    # 73 reviews have neither a review body text, nor a review title

73

In [14]:
# drop reviews with no text
df = df[~(df.reviewText.isnull())]

In [15]:
df.asin.isnull().sum()

0

Let's look at a few actual review texts:

In [16]:
df.reviewText.iloc[np.random.randint(0, len(df))]

"High quality and hides fish flavor well. I ordered this product because I am trying to improve overall health, including vitamin D intake. I also wanted to strengthen my teeth/rebuild enamel. The emulisfied fermented cod liver oil (FCLO) is a little seperated and lumpy when it arrives. You need to mix it with a popsicle stick (or something else, I recommend the popsicle stick because then you can throw it away). Once mixed, it is more of a gel. The peppermint flavor is very strong, so it hides the fish flavor quite well. I keep it in the fridge, and it is better cold.The best way to take it is at night (that way if you get fish burps, you aren't aware) with a shot of chocolate milk. The milk washes away any trace of the oil. Water keeps it in your mouth.I have been taking this for about a month. I have noticed a visable improvement in the enamel of my teeth."

In [17]:
df.reviewText.iloc[np.random.randint(0, len(df))]

'Not what I hoped. The product itself is quality, but oregano oil does not stand up to the claims I have seen so many places. It has a strong odor (of oregano) and can be off putting.'

In [18]:
df.reviewText.iloc[np.random.randint(0, len(df))]

"I'm assuming that this is excellent!. It is hard to review a vitamin or mineral.  If you don't choke on them, if they go down without a hard gulp, if there is no burp afterwards...it works for me.  So, this works for me!"

In [19]:
df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584829 entries, 0 to 584901
Data columns (total 2 columns):
asin          584829 non-null object
reviewText    584829 non-null object
dtypes: object(2)
memory usage: 13.4+ MB


## Pre-processing for LDA

In [20]:
text = list(df.reviewText.values)    # make an iterable to store only the review text

In [21]:
len(text)

584829

In [22]:
# look at a few sample reviews
for rev in text[:4]:
    print(rev, '\n')

Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt around. Good product, good price, good results. 

Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This company promises 99% purity and has fast shipping and good communication. I can't comment on the quality of product because I'm not a chemist but they seem to be legitimate. 

Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspiring. Doing some research on the Internet, it is indicated that taking resveratrol in lozenge form is preferable as it is broken down by stomach acids.  The ez-melt formula recommended in a

In [23]:
nlp = spacy.load('en')

The helper functions below are from:

http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb

In [24]:
for review in tqdm(text):
    parsed_review = nlp(review)
    unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]

 10%|▉         | 57158/584829 [15:03<2:19:00, 63.26it/s]

KeyboardInterrupt: 

 10%|▉         | 57158/584829 [15:20<2:21:41, 62.07it/s]

In [27]:
def lemmatized_sentence_corpus(text_seq):
    """Generator function to use spaCy to parse reviews, 
    lemmatize the text, and yield sentences.
    Parameters:
        text_seq: a sequence (list) of text (e.g. reviews) to parse.
    """
    for parsed_review in nlp.pipe(text_seq, batch_size=1000, n_threads=4):
        for sent in parsed_review.sents:
            # yield makes this process memory efficient
            yield u' '.join([token.lemma_ for token in sent if not (token.is_space or token.is_punct)])

In [28]:
%%time
unigram_sentences_all = [sentence for sentence in lemmatized_sentence_corpus(text)]

KeyboardInterrupt: 

In [ ]:
for parsed_review in tqdm(nlp.pipe(text, batch_size=2000, n_threads=8)):
    unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]


0it [00:00, ?it/s]
1it [00:18, 18.37s/it]
1707it [00:18, 92.44it/s]
2219it [00:38, 57.71it/s]
3825it [00:38, 99.22it/s]

In [ ]:
unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]

In [ ]:
%%time

for parsed_review in nlp.pipe(text, batch_size=1000, n_threads=4):

    # lemmatize the text, removing punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]

    # apply the first-order and second-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]

    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review if term not in spacy.en.STOPWORDS]

    # write the transformed review as a line in the new file
    trigram_review = u' '.join(trigram_review)
    f.write(trigram_review + '\n')

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

### Stop Word Removal